Import needed libraries

In [1]:
import os
import sys
from pathlib import Path

sys.path.append("..")

from utils.llm_client import LLMClient
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.login_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model


In [5]:
model = pick_model("groq", "reason")
llm_client = LLMClient("groq", model)

with open("../data/Incidents.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    headers = [h for h in lines[0].split("|")]

records = []
for line in lines[1:]:
    if not line.strip():
        continue
    values = [v.strip() for v in line.split("|")]
    record = dict(zip(headers, values))
    records.append(record)


prompt_text, spec = render(
    "cot_reasoning.v2",
    role="SOS call analyzer",
    problem = records,
    logic = "Assign a Priority Score (1-10) based on Age Need and Urgency.Base Score: 5, +2 if Age > 60 or < 5, +3 if Need == Rescue (Life Threat), +1 if Need == Insulin/Medicine ",
    output = "Score: {score}/10"
)

messages = [{"role": "user", "content": prompt_text}]
response = llm_client.chat(messages, temperature=1.0)

response_text = response["text"]
print(response_text)

## Reasoning Steps:

1. **Define Base Score**: The base score for all SOS calls is 5.
2. **Age Adjustment**: Add 2 points if the age is greater than 60 or less than 5.
3. **Urgency Adjustment**: 
   - Add 3 points if the main need is 'Rescue' (life-threatening situation).
   - Add 1 point if the main need is 'Insulin/Medicine'.
4. **Calculate Priority Score**: Sum the base score and any applicable adjustments.

## Apply to Each Case:

### Case 1:
- **ID**: 1, **Age**: 20-40, **Need**: Water, **Urgency**: No specific urgency mentioned.
- **Score**: Base Score (5) = 5. No adjustments needed.
- **Priority Score**: 5/10

### Case 2:
- **ID**: 2, **Age**: 75, **Need**: Insulin, **Urgency**: Missed dose, feeling faint.
- **Score**: Base Score (5) + Age Adjustment (+2) + Need Adjustment (+1) = 5 + 2 + 1 = 8.
- **Priority Score**: 8/10

### Case 3:
- **ID**: 3, **Age**: 10, 35, **Need**: Rescue, **Urgency**: Life-threatening situation.
- **Score**: Base Score (5) + Need Adjustment (+3) = 5 + 3

In [4]:
model = pick_model("groq", "reason")
llm_client = LLMClient("groq", model)

with open("../data/Incidents.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    headers = [h for h in lines[0].split("|")]

records = []
for line in lines[1:]:
    if not line.strip():
        continue
    values = [v.strip() for v in line.split("|")]
    record = dict(zip(headers, values))
    records.append(record)


prompt_text, spec = render(
    "tot_reasoning.v2",
    role="SOS call analyzer",
    branches= 3,
    problem = records,
    branching_logic = "Branch 1: Save the highest score first (Greedy),Branch 2: Save closest first (speed), Branch 3: Save furthest first (Logistics).",
)

messages = [{"role": "user", "content": prompt_text}]
response = llm_client.chat(messages, temperature=1.0)

response_text = response["text"]
print(response_text)

### Solution Paths for SOS Call Analyzer

Given the problem, we have three distinct solution paths based on different priorities:

1. **Branch 1: Save the highest score first (Greedy)**
2. **Branch 2: Save closest first (speed)**
3. **Branch 3: Save furthest first (Logistics)**

Let's analyze each path.

### Branch 1: Save the highest score first (Greedy)

**Hypothesis:** Prioritize SOS calls based on a scoring system that considers the urgency and vulnerability of the situation.

**Steps:**
- Assign scores based on urgency and vulnerability:
  - **Urgency Score (US):** 
    - Main Need: Water (2), Insulin (3), Rescue (1)
  - **Vulnerability Score (VS):** 
    - Ages: 20-40 (1), 75 (3), 10, 35 (2)
  - **Total Score (TS) = US + VS + People**
- Calculate scores for each SOS call and sort them in descending order.

**Intermediate Check:**
- **SOS 1:** Main Need = Water, Ages = 20-40, People = 4, Score = 2 (US) + 1 (VS) + 4 (People) = 7
- **SOS 2:** Main Need = Insulin, Ages = 75, People =